In [1]:
import pandas as pd
import os
import numpy as np
from scipy import signal
from scipy.stats import kurtosis, skew
import matplotlib.pyplot as plt
from tqdm import tqdm
from natsort import natsorted
from librosa.feature import mfcc

## Analysis of the EDA signal

In [2]:
import pyphysio as ph
import pyphysio.filters.Filters as flt
import pyphysio.estimators.Estimators as est
import pyphysio.indicators.TimeDomain as td_ind
import pyphysio.indicators.FrequencyDomain as fd_ind

Please cite:
Bizzego et al. (2019) 'pyphysio: A physiological signal processing library for data science approaches in physiology', SoftwareX


In [3]:
def initialize_signal(signal, fs, s_type):
    
    # creazione di un segnale con una fs fissata
    signal = ph.EvenlySignal(values = signal, sampling_freq = fs, signal_type = s_type)
    #signal.plot('r')
    
    # resampling
    signal_resampled = signal.resample(fout=32) # fout: sampling frequency for resampling
    signal = signal_resampled
    #signal.plot('.')
    
    # filtering
    signal_filt = flt.IIRFilter(fp=0.8, fs = 1.1, ftype='ellip')(signal)
    signal = signal_filt
    #signal.plot('g')
    
    # phasic extraction
    driver = est.DriverEstim()(signal)
    phasic, tonic, _ = ph.PhasicEstim(delta=0.02)(driver)
    signal = phasic
    #phasic.plot('b')
    
    # min-max normalization [0,1]
    #signal_normalized = (phasic - np.min(phasic))/np.ptp(phasic)
    #phasic = signal_normalized
    
    return phasic

### Functions for the features

In [4]:
# time domain

def td_median(signal):
    td_median = td_ind.Median()
    td_median_ = td_median(signal)
    #print('median: ', td_median_)
    return td_median_

def td_sum(signal):
    # sum of the values in the signal
    td_sum = td_ind.Sum()
    td_sum_ = td_sum(signal)
    #print('sum: ', td_sum_)
    return td_sum_

def td_AUC(signal):
    # AUC: area under the curve of the signal
    td_AUC = td_ind.AUC()
    td_AUC_ = td_AUC(signal)
    #print('AUC: ', td_AUC_)
    return td_AUC_

def td_RMSSD(signal):
    # RMSSD: square root of the mean of the squared 1st order discrete differences
    td_RMSSD = td_ind.RMSSD()
    td_RMSSD_ = td_RMSSD(signal)
    #print('RMSSD: ', td_RMSSD_)
    return td_RMSSD_

def td_SDSD(signal):
    # SDSD: standard deviation of the 1st order discrete differences
    td_SDSD = td_ind.SDSD()
    td_SDSD_ = td_SDSD(signal)
    #print('SDSD: ', td_SDSD_)
    return td_SDSD_

# frequency domain

def fd_powerinband(signal,f_min, f_max):
    # estimation of the power in a given frequency band
    fd_powerinband = fd_ind.PowerInBand(interp_freq=4, freq_max=f_max, freq_min=f_min, method = 'fft') # create the indicator
    fd_powerinband_ = fd_powerinband(signal.resample(4)) # resampling needed to compute PSD
    return fd_powerinband_

def fd_peakinband(signal,f_min, f_max):
    # estimation of the peak frequency in a given frequency band
    fd_peakinband = fd_ind.PeakInBand(interp_freq=4, freq_max=f_max, freq_min=f_min, method = 'fft') # create the indicator
    fd_peakinband_ = fd_peakinband(signal.resample(4)) # resampling needed to compute PSD
    return fd_peakinband_

# EDA Static features extraction

In [11]:
def extract_EDA_static_features(path_EDA, path_VA):
    
    feature_set = pd.DataFrame()
    count = 1
    
    # for every .mp3 file get a set of features
    for csv_file in tqdm(natsorted(os.listdir(path_EDA))):
        if csv_file.endswith(".csv"):
    
            file_name = os.path.basename(csv_file)
            id = file_name.split('_')[0] # music_ID
            data = pd.read_csv(path_EDA + '/' + file_name, header = None)
            VA_std = pd.read_csv(path_VA + '/static_annotations_std.csv', header = None)
            VA_mean = pd.read_csv(path_VA + '/static_annotations.csv', header = None)
        
            # There are some EDA files with no VA data, so exclude them
            # due to data validation, see PMEmo paper for more details
            if id != VA_std.iloc[count][0]:
                continue

            # save VA values for EDA file
            v_std = VA_std.iloc[count][2]
            a_std = VA_std.iloc[count][1]
            v_mean = VA_mean.iloc[count][2]
            a_mean = VA_mean.iloc[count][1]
            count = count + 1
            
            feature = {}

            # cicle over all the subjects in one EDA file
            for i in range(1,len(data.loc[0])):
                
                subject_ID = data.iloc[0][i]
                
                s = data.iloc[:][i].to_numpy()
                eda_data = np.delete(s,[0])
                
                fs = 50
                phasic = initialize_signal(eda_data, fs = fs, s_type = 'eda')
                f_phasic = np.abs(np.fft.fft(phasic))

                feature['music_ID'] = id
                feature['subject_ID'] = int(subject_ID)
                feature['t_mean'] = np.mean(phasic)
                feature['t_std'] = np.std(phasic)
                feature['t_kurt'] = kurtosis(phasic)
                feature['t_skew'] = skew(phasic)
                feature['t_min'] = phasic.min()
                feature['t_max'] = phasic.max()
                feature['t_range'] = phasic.max()-phasic.min()
                feature['t_median'] = td_median(phasic)
                feature['t_sum'] = td_sum(phasic)
                feature['t_AUC'] = td_AUC(phasic)
                feature['t_RMSSD'] = td_RMSSD(phasic)
                feature['t_SDSD'] = td_SDSD(phasic)
                feature['ZCR'] = ((phasic[:-1] * phasic[1:]) < 0).sum()
                feature['f_mean'] = np.mean(f_phasic)
                feature['f_std'] = np.std(f_phasic)
                feature['f_kurt'] = kurtosis(f_phasic)
                feature['f_skew'] = skew(f_phasic)
                feature['f_min'] = f_phasic.min()
                feature['f_max'] = f_phasic.max()
                feature['f_range'] = f_phasic.max()-f_phasic.min()
                
                # power inband and peak inband, in ranges 0-0.1-0.2-0.3-0.4-0.5 as in PMEmo
                feature['power_inband_1'] = fd_powerinband(phasic,0,0.1)
                feature['power_inband_2'] = fd_powerinband(phasic,0.1,0.2)
                feature['power_inband_3'] = fd_powerinband(phasic,0.2,0.3)
                feature['power_inband_4'] = fd_powerinband(phasic,0.3,0.4)
                feature['power_inband_5'] = fd_powerinband(phasic,0.4,0.5)
                
                feature['peak_inband_1'] = fd_peakinband(phasic,0,0.1)
                feature['peak_inband_2'] = fd_peakinband(phasic,0.1,0.2)
                feature['peak_inband_3'] = fd_peakinband(phasic,0.2,0.3)
                feature['peak_inband_4'] = fd_peakinband(phasic,0.3,0.4)
                feature['peak_inband_5'] = fd_peakinband(phasic,0.4,0.5)
                
                n_mfcc = 12
                mfccCoefficients = mfcc(phasic, sr=32, n_mfcc=n_mfcc)
                for i in range(n_mfcc):
                    mfccCoef = mfccCoefficients[i,:]
                    feature[f'meanMFCC[{i}]'] = np.mean(mfccCoef)
                    feature[f'stdMFCC[{i}]'] = np.std(mfccCoef)
                    feature[f'medianMFCC[{i}]'] = np.median(mfccCoef)
                    feature[f'kurtMFCC[{i}]'] = kurtosis(mfccCoef.reshape(-1,))
                    feature[f'skewMFCC[{i}]'] = skew(mfccCoef.reshape(-1,))                
                
                feature_set = feature_set.append(pd.DataFrame(data=feature, index=[0]))
                
                # [0,1] normalization for every column
                #for n in range(2,len(feature_set.columns)):
                #    a = feature_set.iloc[:,n]
                #    b = (a - np.min(a)) / np.ptp(a)
                #    feature_set.update(b)
                
    return feature_set

In [12]:
#path_EDA = '/Users/gioelepozzi/Desktop/MasterThesis/code/eda_feature_extraction/data'
path_EDA = '/Users/gioelepozzi/Desktop/data/EDA'
path_VA = '/Users/gioelepozzi/Desktop/data/annotations'

static = extract_EDA_static_features(path_EDA, path_VA)

100%|██████████| 795/795 [05:55<00:00,  2.24it/s]


In [216]:
# Converting Dataframe into CSV and JSON file

static.to_csv('static_features_EDA.csv', index=False)
#static.to_json('static_features_EDA.json')

# EDA dynamic features extraction

In [5]:
def window_with_overlap(a, window, stride):
    nrows = ((a.size-window)//stride)+1
    n = a.strides[0]
    # create a view into the array a with the given shape and strides
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,window), strides=(stride*n,n))

In [6]:
def extract_EDA_dynamic_features(path_EDA, path_VA, window, stride):
    
    feature_set = pd.DataFrame()
    count = 1
    
    # for every .mp3 file get a set of features
    for csv_file in tqdm(natsorted(os.listdir(path_EDA))):
        if csv_file.endswith(".csv"):
    
            file_name = os.path.basename(csv_file)
            id = file_name.split('_')[0] # music_ID
            data = pd.read_csv(path_EDA + '/' + file_name, header = None)
            VA_std = pd.read_csv(path_VA + '/static_annotations_std.csv', header = None)
            VA_mean = pd.read_csv(path_VA + '/static_annotations.csv', header = None)
        
            # There are some EDA files with no VA data, so exclude them
            # due to data validation, see PMEmo paper for more details
            if id != VA_std.iloc[count][0]:
                continue

            # save VA values for EDA file
            v_std = VA_std.iloc[count][2]
            a_std = VA_std.iloc[count][1]
            v_mean = VA_mean.iloc[count][2]
            a_mean = VA_mean.iloc[count][1]
            count = count + 1
            
            # cicle over all the subjects in one EDA file
            for i in range(1,len(data.loc[0])):
                
                subject_ID = data.iloc[0][i]
                
                s = data.iloc[:][i].to_numpy()
                eda_data = np.delete(s,[0])
                
                fs = 50
                phasic = initialize_signal(eda_data, fs = fs, s_type = 'eda')

                sr=32 # due to resampling in initialize_signal
                times = np.arange(len(phasic))/sr
                frames = window_with_overlap(phasic, window, stride)
                time_frames = window_with_overlap(times, window, stride)

                for fidx, frame in enumerate(frames):
                    
                    feature = {}
                
                    frame_time = fidx*0.5+1
                    times = time_frames[fidx]
                    #t_frame = initialize_signal(frame, fs = 1, s_type = 'eda')
                    f_frame = np.abs(np.fft.fft(frame))
                    
                    feature['music_ID'] = id
                    feature['subject_ID'] = int(subject_ID)
                    feature['frame'] = frame_time
                    
                    feature['t_mean'] = np.mean(frame)
                    feature['t_std'] = np.std(frame)
                    feature['t_kurt'] = kurtosis(frame)
                    feature['t_skew'] = skew(frame)
                    feature['t_min'] = frame.min()
                    feature['t_max'] = frame.max()
                    feature['t_range'] = frame.max()-frame.min()
                    #feature['t_median'] = td_median(t_frame)
                    #feature['t_sum'] = td_sum(t_frame)
                    #feature['t_AUC'] = td_AUC(t_frame)
                    #feature['t_RMSSD'] = td_RMSSD(t_frame)
                    #feature['t_SDSD'] = td_SDSD(t_frame)
                    feature['ZCR'] = ((frame[:-1] * frame[1:]) < 0).sum()
                    feature['f_mean'] = np.mean(f_frame)
                    feature['f_std'] = np.std(f_frame)
                    feature['f_kurt'] = kurtosis(f_frame)
                    feature['f_skew'] = skew(f_frame)
                    feature['f_min'] = f_frame.min()
                    feature['f_max'] = f_frame.max()
                    feature['f_range'] = f_frame.max()-f_frame.min()

                    # power inband and peak inband, in ranges 0-0.1-0.2-0.3-0.4-0.5 as in PMEmo
                    #feature['power_inband_1'] = fd_powerinband(t_frame,0,0.1)
                    #feature['power_inband_2'] = fd_powerinband(t_frame,0.1,0.2)
                    #feature['power_inband_3'] = fd_powerinband(t_frame,0.2,0.3)
                    #feature['power_inband_4'] = fd_powerinband(t_frame,0.3,0.4)
                    #feature['power_inband_5'] = fd_powerinband(t_frame,0.4,0.5)

                    #feature['peak_inband_1'] = fd_peakinband(t_frame,0,0.1)
                    #feature['peak_inband_2'] = fd_peakinband(t_frame,0.1,0.2)
                    #feature['peak_inband_3'] = fd_peakinband(t_frame,0.2,0.3)
                    #feature['peak_inband_4'] = fd_peakinband(t_frame,0.3,0.4)
                    #feature['peak_inband_5'] = fd_peakinband(t_frame,0.4,0.5)

                    n_mfcc = 12
                    mfccCoefficients = mfcc(frame, sr=32, n_mfcc=n_mfcc)
                    for i in range(n_mfcc):
                        mfccCoef = mfccCoefficients[i,:]
                        feature[f'meanMFCC[{i}]'] = np.mean(mfccCoef)
                        feature[f'stdMFCC[{i}]'] = np.std(mfccCoef)
                        feature[f'medianMFCC[{i}]'] = np.median(mfccCoef)
                        feature[f'kurtMFCC[{i}]'] = kurtosis(mfccCoef.reshape(-1,))
                        feature[f'skewMFCC[{i}]'] = skew(mfccCoef.reshape(-1,))                
                        
                
                    feature_set = feature_set.append(pd.DataFrame(data=feature, index=[0]))
                
                    # [0,1] normalization for every column
                    #for n in range(2,len(feature_set.columns)):
                    #    a = feature_set.iloc[:,n]
                    #    b = (a - np.min(a)) / np.ptp(a)
                    #    feature_set.update(b)
                
    return feature_set

In [7]:
#path_EDA = '/Users/gioelepozzi/Desktop/MasterThesis/code/eda_feature_extraction/data'
path_EDA = '/Users/gioelepozzi/Desktop/data/EDA'
path_VA = '/Users/gioelepozzi/Desktop/data/annotations'

dynamic = extract_EDA_dynamic_features(path_EDA, path_VA, 50, 25)

100%|██████████| 795/795 [16:07:39<00:00, 73.03s/it]   


In [8]:
# Converting Dataframe into CSV and JSON file

dynamic.to_csv('dynamic_features_EDA.csv', index=False)
#static.to_json('dynamic_features_EDA.json')